# 第8章 : 拡散モデルの理論

## VAEから拡散モデルへ
VAEでは潜在変数が1つである。これを複数個にして階層化したモデルを階層化VAEという。潜在変数が$T$個である階層化VAEは、以下の式で表される

$$
\begin{align}
p_{\boldsymbol{\theta}_T}\left(\boldsymbol{z}_{T} \mid \boldsymbol{z}_{T-1}\right),\cdots, p_{\boldsymbol{\theta}_2}\left(\boldsymbol{z}_1 \mid \boldsymbol{z}_2\right),p_{\boldsymbol{\theta}_1}\left(\boldsymbol{x} \mid \boldsymbol{z}_1\right) \\
q_{\boldsymbol{\phi}_T}\left(\boldsymbol{z}_{T-1} \mid \boldsymbol{z}_{T}\right), \cdots, q_{\boldsymbol{\phi}_2}\left(\boldsymbol{z}_2 \mid \boldsymbol{z}_1\right),q_{\boldsymbol{\phi}_1}\left(\boldsymbol{z}_1 \mid \boldsymbol{x}\right) \\

\end{align}
$$

これはマルコフ過程である。

これは全部で$2T$個のニューラルネットワークが必要だが、拡散モデルでは以下の方法によって1つのニューラルネットワークだけで済む
- 観測変数と潜在変数の次元を同じにする
- エンコーダは固定の正規分布によるノイズを追加する
よって、以下のように表される

$$
\begin{align}
p_{\boldsymbol{\theta}}\left(\boldsymbol{z}_T \mid \boldsymbol{z}_{T-1}\right),\cdots, p_{\boldsymbol{\theta}}\left(\boldsymbol{z}_1 \mid \boldsymbol{z}_2\right),p_{\boldsymbol{\theta}}\left(\boldsymbol{x} \mid \boldsymbol{z}_1\right) \\
q\left(\boldsymbol{z}_{T-1} \mid \boldsymbol{z}_{T}\right), \cdots, q\left(\boldsymbol{z}_2 \mid \boldsymbol{z}_1\right),q\left(\boldsymbol{z}_1 \mid \boldsymbol{x}\right) \\

\end{align}
$$

このように全ての潜在変数の次元が等しいため1つのパラメータ$\theta$でかける。

上の式を逆拡散過程、下の式を拡散過程という。

## 拡散過程

$$
\begin{align}
q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1} \right) = \mathcal{N}\left( \boldsymbol{x}_t; \sqrt{1 - \beta_t} \boldsymbol{x}_{t-1}, \beta_t \boldsymbol{I} \right)
\end{align}
$$

このとき、
$$
p\left(\boldsymbol{x}_T \right) \approx \left(\boldsymbol{x}_T; \boldsymbol{0},\boldsymbol{I}\right)
$$

が成り立つ。

拡散過程の式は変数変換トリックを使うと
$$
\boldsymbol{\epsilon} \sim \mathcal{N}\left(\boldsymbol{\epsilon};\boldsymbol{0},\boldsymbol{I}\right) \\
\boldsymbol{x}_t = \sqrt{1-\beta_t}\boldsymbol{x}_{t-1} + \sqrt{\beta_t}\boldsymbol{\epsilon}
$$
とかける。
これは、前データの$\boldsymbol{x}_{t-1}$を少しだけスケールダウンして小さなノイズを加えるということである。スケールダウンするのはノイズ加算後の分散が大きくなることを防ぐためである。

## 逆拡散過程
潜在変数の次元が全て等しいことから1つのニューラルネットワークだけが必要になる。入力はノイズ付与された画像$\boldsymbol{x}_t$と時刻$t$である。
$$
\begin{align}
\hat{\boldsymbol{x}}_{t-1} = \mathrm{NeuralNet}\left(\boldsymbol{x}_t,t;\boldsymbol{\theta}\right) \\
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right) = \mathcal{N}\left(\boldsymbol{x}_{t-1};\hat{\boldsymbol{x}}_{t-1},\boldsymbol{I}\right)
\end{align}
$$

ここでは単純化のためにNNによる出力を平均だけにして、共分散行列を$\boldsymbol{I}$としている。

## ELBO
拡散モデルの学習はVAEと同じく対数尤度の下界であるELBOを最大化することを考える。
拡散モデルのELBOは、VAEと同じく導出できる。VAEでは、
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta},\boldsymbol{\phi}\right) &= \int q_{\phi}\left(\boldsymbol{z} \mid \boldsymbol{x}\right)\log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x},\boldsymbol{z}\right)}{q_{\boldsymbol{\phi}}\left(\boldsymbol{z} \mid \boldsymbol{x} \right)} d\boldsymbol{z} \\
&= \mathbb{E}_{q_{\boldsymbol{\phi}}\left(\boldsymbol{z}\mid\boldsymbol{x}\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x},\boldsymbol{z}\right)}{q_{\boldsymbol{\phi}\left(\boldsymbol{z} \mid \boldsymbol{x} \right)}} \right]
\end{align}
$$
であった。そのため、拡散モデルでは
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta}\right) = \mathbb{E}_{q\left(\boldsymbol{x}_1,\boldsymbol{x}_2,\cdots,\boldsymbol{x}_T \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_0, \boldsymbol{x}_1, \boldsymbol{x}_2, \cdots, \boldsymbol{x}_T\right)}{q \left( \boldsymbol{x}_1, \boldsymbol{x}_2, \cdots, \boldsymbol{x}_T \mid \boldsymbol{x}_0\right)} \right]
\end{align}
$$

とかける。ここで、簡略化のために
$$
\boldsymbol{x}_{0:T} = \boldsymbol{x}_0, \boldsymbol{x}_1, \cdots, \boldsymbol{x}_T
$$
とかくことにする。
よって、

$$
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta}\right) = \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)}{q \left( \boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \right]
$$
とかける。

### ELBOの式展開
$p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)$は乗法定理とマルコフ性によって次の式でかける

$$
\begin{align}
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right) &= p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_0 \mid \boldsymbol{x}_1\right)  p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_1 \mid \boldsymbol{x}_2\right) \dots  p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{T-1} \mid \boldsymbol{x}_T\right) p \left(\boldsymbol{x}_T\right) \\
&= p\left(\boldsymbol{x}_T \right) \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}
\end{align}
$$
ここで、$p\left(\boldsymbol{x}_T\right)$ は $\mathcal{N}\left(\boldsymbol{x}_T ; \boldsymbol{0}, \boldsymbol{I}\right)$ である。

拡散過程についても同様に

$$
q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)=\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}
$$

とかける。

これより、
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x}_0;\boldsymbol{\theta}\right) &= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)}\left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)}{q \left( \boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \right] \\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p\left(\boldsymbol{x}_T \right) \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}}{\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}}\right]\\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[\log \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)} + \underbrace{\log \frac{ p\left(\boldsymbol{x}_T \right)}{\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}}}_{\boldsymbol{\theta}\text{を含まない項}}\right]
\end{align}
$$

$\boldsymbol{\theta}$に関してELBOを最適化するため第2項は無視できて、

$$
\begin{align}
J \left(\boldsymbol{\theta} \right) &:= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[\log \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}\right] \\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \sum_{t=1}^{T}{ \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}\right]
\end{align}
$$